In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator 
from keras import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Rescaling
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.nn import softmax
# import keras_tuner as kt
auc_roc_metric_ROC = tf.keras.metrics.AUC(curve='ROC')

2023-04-25 00:12:43.594918: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 00:12:43.885892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 00:12:45.010718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-25 00:12:47.069277: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at htt

In [2]:
# %config Completer.use_jedi = False
import random as python_random

def reset_seeds():
   np.random.seed(10) 
   python_random.seed(10)
   tf.random.set_seed(10)

reset_seeds()

# Articles 
***
# https://www.tensorflow.org/tutorials/images/classification

https://towardsdatascience.com/the-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480

https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

https://www.analyticsvidhya.com/blog/2021/08/quick-start-with-tensorflow-callbacks/

https://towardsdatascience.com/how-to-choose-the-right-activation-function-for-neural-networks-3941ff0e6f9c

https://towardsdatascience.com/deep-learning-which-loss-and-activation-functions-should-i-use-ac02f1c56aa8

https://medium.com/featurepreneur/data-augmentation-using-keras-preprocessing-layers-6cdc7d49328e

https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7

https://towardsdatascience.com/mastering-tensorflow-variables-in-5-easy-step-5ba8062a1756

https://towardsdatascience.com/how-to-choose-the-best-keras-pre-trained-model-for-image-classification-b850ca4428d4|


In [3]:
# # To Import Data from google drive (authentication needed)
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [ ]:
# Display an background image
from PIL import Image
with Image.open( '/home/abhishek/triplet_codons/images_classification_data_1st/images_classification_data_axion_classification_pT_size305/background_bb_constituents_data_images_pT/pT_0.png') as im:
    fig = px.imshow(im)
    fig.update_layout(title='An background image', width=500)
    fig.show()
    images_size = im.size
    print( 'Size of the Images', im.size )

In [ ]:
# Display an signal image
from PIL import Image
with Image.open( '/home/abhishek/triplet_codons/images_classification_data_1st/images_classification_data_axion_classification_pT_size305/signal_axion_constituents_data_images_pT/pT_0.png' ) as im:
    fig = px.imshow(im)
    fig.update_layout(width=500)
    fig.show()
    images_size = im.size
    print( 'Size of the Images', im.size )

In [10]:
# # Importing Dataset
# train = tf.keras.utils.image_dataset_from_directory( directory='/home/abhishek/triplet_codons/images_classification_data_1st/images_classification_data_axion_classification_pT_size305/',
#                                             shuffle=True,seed=10,color_mode='rgb',image_size=(305,305), 
#                                             batch_size=64, validation_split=0.2,subset="training" )
# val = tf.keras.utils.image_dataset_from_directory( directory='/home/abhishek/triplet_codons/images_classification_data_1st/images_classification_data_axion_classification_pT_size305/',
#                                             shuffle=True,seed=10,color_mode='rgb',image_size=(305,305),
#                                             batch_size=64, validation_split=0.2,subset="validation" )

train = tf.keras.utils.image_dataset_from_directory( directory='/home/abhishek/Desktop/deltaeta_invmass_images_classification/images_norep_nocuts/',
                                            shuffle=True,seed=10,color_mode='grayscale',image_size=(2921,2921), 
                                            batch_size=64, validation_split=0.2,subset="training" )
val = tf.keras.utils.image_dataset_from_directory( directory='/home/abhishek/Desktop/deltaeta_invmass_images_classification/images_norep_nocuts/',
                                            shuffle=True,seed=10,color_mode='grayscale',image_size=(2921,2921),
                                            batch_size=64, validation_split=0.2,subset="validation" )


Found 6000 files belonging to 2 classes.
Using 4800 files for training.
Found 6000 files belonging to 2 classes.
Using 1200 files for validation.


In [11]:
class_names = train.class_names
no_of_classes = len( class_names )
class_names

['background', 'signal']

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train = train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)

## Notation:
 * **24C5** --> convolution layer (24 feature, 5x5 filter and stride 1)
 * **24C5S2** --> convolution layer (24 feature, 5x5 filter and stride 2)
 * **MP2S2** --> max pooling (2x2 filter and stride 2) ( Default (2,2))
 * **256D** --> fully connected dense layer (256 units) 
 

## Step 1: Estimate Number of Convolution Layers required
* 1)  256*256*3-50C3MP2-F-150D-4D-Compile
* 2)  256*256*3-50C3MP2-30C3MP2-F-150D-4D-Compile
* 3)  256*256*3-50C3MP2-30C3MP2-20C3MP2-F-150D-4D-Compile
* 4)  256*256*3-50C3MP2-40C3MP2-30C3MP2-20C3MP2-F-150D-4D-Compile

In [13]:
model_1 = Sequential()
model_1.add( Conv2D( filters=5 ,kernel_size=3, padding='same', activation='relu', input_shape=(2921,2921,1) ) )
model_1.add( MaxPooling2D() ) 

model_1.add( Flatten() )
model_1.add( Dense(units=256,activation='relu') )
model_1.add( Dense(units=1, activation='sigmoid') )
model_1.compile( optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy',auc_roc_metric_ROC] )
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2921, 2921, 5)     50        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1460, 1460, 5)    0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 10658000)          0         
                                                                 
 dense (Dense)               (None, 256)               2728448256
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2,728,448,563
Trainable params: 2,728,448,563
Non-trainable params: 0
______________________________________

In [14]:
model_1.fit( train, validation_data=val, epochs=5 )

Epoch 1/5


2023-04-25 00:15:24.240824: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [4800]
	 [[{{node Placeholder/_0}}]]
2023-04-25 00:15:24.241185: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [4800]
	 [[{{node Placeholder/_4}}]]
2023-04-25 00:15:36.933724: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 7 of 1000
2023-04-25 00:15:47.688897: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 23 of 1000


: 

: 

## Step 8: Add Data Augmentation

## Step 9: Adding Callback

In [ ]:
model = Sequential()
model.add( layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(305,305,3)))
model.add( layers.experimental.preprocessing.RandomRotation(0.2) )
# model.add( layers.experimental.preprocessing.RandomCrop(width=230,height=230) )
model.add( layers.experimental.preprocessing.RandomZoom(0.2) )
model.add( Conv2D( filters=8, kernel_size=3, padding='same', activation='relu') )
model.add( BatchNormalization() )
model.add( MaxPooling2D() )
            
model.add( Conv2D( filters=16, kernel_size=3, padding='same', activation='relu' ) )
model.add( BatchNormalization() )
model.add( MaxPooling2D() )
model.add( Dropout(0.1))
            
model.add( Conv2D( filters=32, kernel_size=3, padding='same', activation='relu' ) )
model.add( BatchNormalization() )
model.add( MaxPooling2D() )
model.add( Dropout(0.1))

model.add( Flatten() )
model.add( Dense(units=64,activation='relu') )
model.add( BatchNormalization() )
model.add( Dropout(0.2))
model.add( Dense(units=1, activation='sigmoid') )
model.compile( optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',auc_roc_metric_ROC])
model.summary()

In [ ]:
callback = EarlyStopping(monitor='loss', patience=2)
history = model.fit( train, validation_data=val, epochs=20, callbacks=[callback])

## Predictions

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.linspace(1,15,15)
epochs

In [ ]:
fig = go.Figure(data=go.Scatter( x=epochs, y=loss, name='loss' ))
fig.add_trace( go.Scatter( x=epochs, y=val_loss, name='val_loss' ) )
fig.update_layout( title="Loss", xaxis_title='epochs' )
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter(x=epochs, y=accuracy, name='accuracy' ))
fig.add_trace( go.Scatter( x=epochs, y=val_accuracy, name='val_accuracy' ) )
fig.update_layout( title="Accuracy",xaxis_title='epochs' )
fig.show()

In [ ]:
predictions_test = model.predict(val)

for images,labels in val.take(1):
    for i in range(10):
        score = softmax( predictions_test[i] )
        fig = px.imshow( images[i] )
        fig.update_layout(width=500,title=class_names[labels[i]])
        fig.show()
        print( 'Actaul Class: ', class_names[labels[i]] )
        print('Predicted Class: {} with Percent Confidence:{}'.format(class_names[np.argmax(score)], 100 * np.max(score)))
        if class_names[labels[i]] != class_names[np.argmax(score)] :
            print("Incorrect Prediction")
        else:
            print('Correct Prediction')

In [ ]:
# predictions_val = model.predict(val)

# for images,labels in val.take(1):
#     for i in range(10):
#         score = softmax( predictions_val[i] )
#         fig = px.imshow( images[i] )
#         fig.update_layout(width=500,title=class_names[labels[i]])
#         fig.show()
#         print( 'Actaul Class: ', class_names[labels[i]] )
#         print('Predicted Class: {} with Percent Confidence:{}'.format(class_names[np.argmax(score)], 100 * np.max(score)))
#         if class_names[labels[i]] != class_names[np.argmax(score)] :
#             print("Incorrect Prediction")
#         else:
#             print('Correct Prediction')

In [ ]:
# model.save('./Step_By_Step_Method_Model')

***
***
***

## Creating Some Useful Functions

In [ ]:
# Plotting Loss vs epochs graph, Accuaracy vs epochs graph
def Plot_Loss_Accuracy( history ):
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = np.linspace(1,15,15)
    epochs


    fig = go.Figure(data=go.Scatter( x=epochs, y=loss, name='loss' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_loss, name='val_loss' ) )
    fig.update_layout( title="Loss", xaxis_title='epochs' )
    fig.show()


    fig = go.Figure(data=go.Scatter(x=epochs, y=accuracy, name='accuracy' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_accuracy, name='val_accuracy' ) )
    fig.update_layout( title="Accuracy",xaxis_title='epochs' )
    fig.show()


In [ ]:
# Displaying Images in Validation/Test Set, Model Predictions and Confidence Level
def Prediction_Results( model, data, no_of_images):
    predictions = model.predict(data)

    for images,labels in data.take(1):
        for i in range(no_of_images):
            score = softmax( predictions[i] )
            fig = px.imshow( images[i] )
            fig.update_layout(width=500,title=class_names[labels[i]])
            fig.show()
            print( 'Actaul Class: ', class_names[labels[i]] )
            print('Predicted Class: {} with Percent Confidence:{}'.format(class_names[np.argmax(score)], 100 * np.max(score)))
            if class_names[labels[i]] == class_names[np.argmax(score)] :
                print("Correct Prediction")
            else:
                print('Incorrect Prediction')

***
***

***
***

# Transfer Learning

## Trasnfer Learning using ResNet50

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator 
from keras import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Rescaling
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.nn import softmax
# load_img(r'C:\Users\shirs\Desktop\Image Classification\Cotton Disease Prediction Project-Youtube Krish Naik\train\fresh cotton leaf\d (88)_iaip.jpg')

In [ ]:
model_ResNet50 = Sequential()
model_ResNet50.add( layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(256,256,3)))
model_ResNet50.add( layers.experimental.preprocessing.RandomRotation(0.2) )
# model_ResNet50.add( layers.experimental.preprocessing.RandomCrop(width=230,height=230) )
model_ResNet50.add( layers.experimental.preprocessing.RandomZoom(0.2) )
model_ResNet50.add( layers.RandomContrast(0.2) )
pretrained_model = tf.keras.applications.ResNet50( include_top=False,classes=2,weights='imagenet' )
pretrained_model.trainable = False
model_ResNet50.add( pretrained_model )
model_ResNet50.add( Flatten() )
model_ResNet50.add( Dense( units=256, activation='relu'))
model_ResNet50.add( BatchNormalization() )
# model_ResNet50.add( Dropout(0.1) )
model_ResNet50.add( Dense( units=256, activation='relu'))
model_ResNet50.add( Dense( units=1, activation='sigmoid' ) )
model_ResNet50.summary()

In [ ]:
model_ResNet50.compile( optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',auc_roc_metric_ROC])
history = model_ResNet50.fit( train, validation_data=val, epochs=20 )

In [ ]:
model_ResNet50.save('./TransferLearning_ResNet50_Model')

In [ ]:
Plot_Loss_Accuracy( history=history )

## Transfer Learning using VGG16 Network

In [ ]:
model_VGG16 = Sequential()
pretrained_model = tf.keras.applications.VGG16(include_top=False, classes=2, weights='imagenet' )
pretrained_model.trainable = False
model_VGG16.add( layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(305,305,3)))
model_VGG16.add( layers.experimental.preprocessing.RandomRotation(0.2) )
#model_VGG16.add( layers.experimental.preprocessing.RandomCrop(width=230,height=230) )
model_VGG16.add( layers.experimental.preprocessing.RandomZoom(0.2) )
model_VGG16.add( layers.RandomContrast(0.2) )
model_VGG16.add( pretrained_model )
model_VGG16.add( Flatten() )
model_VGG16.add( Dense( units=512, activation='relu'))
model_VGG16.add( BatchNormalization())
model_VGG16.add( Dense( units=256, activation='relu'))
model_VGG16.add( Dense( units=1, activation='sigmoid' ) )
model_VGG16.summary()

In [ ]:
model_VGG16.compile( optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',auc_roc_metric_ROC])
history = model_VGG16.fit( train, validation_data=val, epochs=20 )

In [ ]:
model_VGG16.save('./TransferLearning_VGG16_Model')

In [ ]:
Plot_Loss_Accuracy( history=history )

### Best Validation Accuracy is provided by ResNet50 Model
### Train Accuracy of VGG16 Model is also good but it takes very long time to complete the training ( 10 minutes per epochs )


***
***

In [ ]:
final_model = tf.keras.models.load_model('./TransferLearning_ResNet50_Model' )